Working on Kinectrics Data

In [2]:
#%% 1. Library setup
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import copy

In [3]:
DATANAME2 = "OfficeTest-Mesh_00.ply"
pcd = o3d.io.read_point_cloud("../Data/" + DATANAME2)

In [4]:
# Print basic info
print(pcd)

# Print number of points
print("Number of points:", np.asarray(pcd.points).shape[0])

# Show the first few points
print("First 5 points:\n", np.asarray(pcd.points)[:5])

PointCloud with 2499875 points.
Number of points: 2499875
First 5 points:
 [[ 6.375779 -7.119489 -1.878987]
 [ 6.371648 -7.118566 -1.881008]
 [ 6.371648 -7.119278 -1.878987]
 [ 6.375779 -7.118566 -1.88167 ]
 [ 6.379911 -7.119558 -1.878987]]


In [5]:
# shift point cloud to bypasss large coordinates approxximation
pcd_center = pcd.get_center()
print("pcd_center:", pcd_center)
print("Type:", type(pcd_center))

pcd_center: [ 4.5261486  -8.87957326 -1.13912606]
Type: <class 'numpy.ndarray'>


In [6]:
pcd.translate(-pcd_center)

PointCloud with 2499875 points.

In [7]:
o3d.visualization.draw_geometries([pcd])

In [8]:
# Random sampling
retained_ratio = 0.2 # keep 20% of the points
sampled_pcd = pcd.random_down_sample(retained_ratio)

In [9]:
o3d.visualization.draw_geometries([sampled_pcd], window_name = "Random Sampling")

In [10]:
# outlier removal
nn = 16 # How many neightbors are considered to calculate the averate distance for a given point
std_multiplier = 10 # threshold based on standard deviation. the lower the number, the more aggressive the filter will be

filtered_pcd, filtered_idx = pcd.remove_statistical_outlier(nn, std_multiplier)

In [11]:
# colour outliers in red
outliers = pcd.select_by_index(filtered_idx, invert=True)
outliers.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([filtered_pcd, outliers])

In [12]:
# voxel sampling
voxel_size = 0.05
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size = voxel_size)

In [13]:
o3d.visualization.draw_geometries([pcd_downsampled])

In [14]:
#normals extraction
nn_distance = 0.05

In [15]:
radius_normals=nn_distance*4
pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

In [16]:
pcd_downsampled.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd_downsampled,outliers])

In [17]:
# RANSAC
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())

In [18]:
distance_threshold = 0.1 # max distance a point can be from the plane model, and still be an inlier
ransac_n = 3 # number of points to sample for generating a plane model
num_iterations = 1000 # number of iterations

In [19]:
plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,ransac_n=3,num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: 0.01x + 0.00y + 1.00z + 0.70 = 0


In [20]:
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

#Paint the clouds
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

#Visualize the inliers and outliers
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [21]:
# multi order RANSAC
# Empty dictionary to hold results of iterations
segment_models={} # Plane parameters
segments={} # Planar regions
max_plane_idx=10

In [22]:
# seperate inliers from outliers. Store inliers in segments dictionary.
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [23]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [24]:
# DBSCAN
epsilon = 0.3
min_cluster_points = 35

In [25]:
rest=sampled_pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(
    distance_threshold=0.1,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=epsilon, min_points=min_cluster_points)) # DBSCAN clustering
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]])
    rest = rest.select_by_index(inliers, invert=True) + segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels== best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i,"/",max_plane_idx,"done.")


/var/folders/37/jl4db58d37v4cx7r8qrqx9vw0000gn/T/ipykernel_35937/184731682.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  best_candidate=int(np.unique(labels)[np.where(candidates== np.max(candidates))[0]])


pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


In [26]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [27]:
o3d.visualization.draw_geometries([rest])

In [28]:
labels = np.array(sampled_pcd.cluster_dbscan(eps=0.1, min_points=10)) # -1 is noise. 0-n are the clusters

In [29]:
# Color DBSCAN clusters
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0  # Noise points in black
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([rest], window_name="DBSCAN Clusters")


In [30]:
voxel_size = 0.5


In [31]:
# Calculate the bounds of the point cloud
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()
print("Point cloud bounds:", min_bound, max_bound)

Point cloud bounds: [-1.2162536  -2.10670774 -0.74399294] [1.9644834  1.76251926 1.23080706]


In [32]:
# Concatenate all RANSAC segments
pcd_ransac = o3d.geometry.PointCloud()
for seg in segments.values():
    pcd_ransac += seg

In [33]:
# Create voxel grids
voxel_grid_structural = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_ransac, voxel_size=voxel_size)
rest.paint_uniform_color([0.1, 0.1, 0.8])
voxel_grid_clutter = o3d.geometry.VoxelGrid.create_from_point_cloud(rest, voxel_size=voxel_size)

o3d.visualization.draw_geometries([voxel_grid_structural, voxel_grid_clutter], window_name="Voxel Grids")


In [34]:
# Function: Fit voxel grid as boolean array
def fit_voxel_grid(points, voxel_size, min_b=False, max_b=False):
    if type(min_b) == bool:
        min_coords = np.min(points, axis=0)
        max_coords = np.max(points, axis=0)
    else:
        min_coords = min_b
        max_coords = max_b
    grid_dims = np.ceil((max_coords - min_coords) / voxel_size).astype(int)
    voxel_grid = np.zeros(grid_dims, dtype=bool)
    indices = ((points - min_coords) / voxel_size).astype(int)
    indices = np.clip(indices, 0, grid_dims - 1)
    voxel_grid[indices[:,0], indices[:,1], indices[:,2]] = True
    return voxel_grid, indices

In [35]:
# Fit voxel grids for RANSAC segments and remaining points
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()

ransac_voxels, idx_ransac = fit_voxel_grid(np.asarray(pcd_ransac.points), voxel_size, min_bound, max_bound)
rest_voxels, idx_rest = fit_voxel_grid(np.asarray(rest.points), voxel_size, min_bound, max_bound)
filled_ransac = np.transpose(np.nonzero(ransac_voxels))
filled_rest = np.transpose(np.nonzero(rest_voxels))

In [36]:
# Compute empty voxels
total = pcd_ransac + rest
total_voxels, _ = fit_voxel_grid(np.asarray(total.points), voxel_size, min_bound, max_bound)
empty_indices = np.transpose(np.nonzero(~total_voxels))

In [37]:
# Export RANSAC segments + rest
xyz_segments = []
for idx, seg in segments.items():
    pts = np.asarray(seg.points)
    N = len(pts)
    labels = np.full((N,1), idx)
    xyz_segments.append(np.hstack((pts, labels)))

rest_labels = np.full((len(rest.points), 1), len(segments))
xyz_segments.append(np.hstack((np.asarray(rest.points), rest_labels)))

np.savetxt("../Results/OfficeSegments.xyz", np.concatenate(xyz_segments), delimiter=";", fmt="%1.9f")
print("Saved OfficeSegments.xyz")

Saved OfficeSegments.xyz


In [38]:
# Voxel Modeling Export
def cube(c, s, compteur=0):
    v1 = c + s/2*np.array([-1,-1,1])
    v2 = c + s/2*np.array([1,-1,1])
    v3 = c + s/2*np.array([-1,1,1])
    v4 = c + s/2*np.array([1,1,1])
    v5 = c + s/2*np.array([-1,1,-1])
    v6 = c + s/2*np.array([1,1,-1])
    v7 = c + s/2*np.array([-1,-1,-1])
    v8 = c + s/2*np.array([1,-1,-1])
    vcube = [v1,v2,v3,v4,v5,v6,v7,v8]
    vert = np.hstack((np.full((8,1),'v'), np.array(vcube)))
    faces = np.array([[1,2,3],[3,2,4],[3,4,5],[5,4,6],
                      [5,6,7],[7,6,8],[7,8,1],[1,8,2],
                      [2,8,4],[4,8,6],[7,1,5],[5,1,3]]) + compteur*8
    faces = np.hstack((np.full((12,1),'f'), faces))
    return np.append(vert, faces, axis=0)

def voxel_modelling(filename, indices, voxel_size):
    voxel_assembly = []
    with open(filename, "w") as f:
        for cpt, idx in enumerate(indices):
            voxel = cube(idx, voxel_size, cpt)
            f.write(f"o voxel_{cpt}\n")
            np.savetxt(f, voxel, fmt='%s')
            voxel_assembly.append(voxel)
    return voxel_assembly

In [39]:
v_ransac = voxel_modelling("../Results/Office_RANSAC_Vox.obj", filled_ransac, 1)
v_rest = voxel_modelling("../Results/Office_Rest_Vox.obj", filled_rest, 1)
v_empty = voxel_modelling("../Results/Office_Empty_Vox.obj", empty_indices, 1)

print("Voxel models exported successfully.")

Voxel models exported successfully.
